In [0]:
# importando biblioteca
import pyspark.pandas as ps

In [0]:
# leitura dos arquivos formato Parquet
df_junto = ps.read_parquet("dbfs:/databricks-results/bronze/*/*/*")
df_junto.head()

,moeda,taxa,data
0,USD,0.201077,2024-02-05
1,GBP,0.160427,2024-02-05
2,EUR,0.187198,2024-02-05
3,JPY,29.889123,2024-02-05
4,CNY,1.431367,2024-02-05


In [0]:
# filtrando moedas de interesse 'USD', 'EUR', 'GBP'
moedas = ['USD', 'EUR', 'GBP']
df_moedas = df_junto[df_junto['moeda'].isin(moedas)]
df_moedas.head()

,moeda,taxa,data
0,USD,0.201077,2024-02-05
1,GBP,0.160427,2024-02-05
2,EUR,0.187198,2024-02-05
8,EUR,0.187288,2024-02-06
14,USD,0.201175,2024-02-07


In [0]:
# convertendo a coluna string data para datetime
df_moedas['data'] = ps.to_datetime(df_moedas['data'])


In [0]:
# pivotando o DataFrame

resultado_taxas_conversao = df_moedas.pivot_table(
    values='taxa',
    index=['data'],
    columns='moeda',
    aggfunc='first'
    ).sort_index(ascending=False)

resultado_taxas_conversao.head()


moeda,EUR,GBP,USD
data,,,
2024-04-04,0.182614,0.156560,0.197914
2024-04-03,0.183063,0.156812,0.198409
2024-04-02,0.183317,0.156959,0.197398
2024-04-01,0.184242,0.157692,0.197820
2024-03-31,0.184771,0.157804,0.199414


In [0]:
# cópia do DataFrame
resultado_valores_reais = resultado_taxas_conversao.copy()


In [0]:
for moeda in moedas:
    resultado_valores_reais[moeda] = resultado_valores_reais[moeda]\
        .apply(
            lambda x: round(1 / x, 4)
)

In [0]:
# resultado da transformação da conversão de moeda e arredondado para 4 casas decimais
resultado_valores_reais

moeda,EUR,GBP,USD
data,,,
2024-04-04,5.4760,6.3873,5.0527
2024-04-03,5.4626,6.3771,5.0401
2024-04-02,5.4550,6.3711,5.0659
2024-04-01,5.4276,6.3415,5.0551
2024-03-31,5.4121,6.3370,5.0147
2024-03-30,5.4163,6.3322,5.0152
2024-03-27,5.3977,6.2976,4.9918
2024-03-25,5.3905,6.2854,4.9737
2024-03-24,5.4063,6.3021,5.0024
